# 수화 동영상 파일 -> 텐서 파일(+ 미디어파이프 처리 영상 파일)

In [8]:
import os
import torch
import mediapipe as mp
import numpy as np
import cv2
import tensorflow as tf
import pandas as pd
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.python.client import device_lib

In [9]:
# 모션 랜드마크를 리스트로 반환
def convert_landmark_to_tensor(left_hand_landmarks, right_hand_landmarks):
    # 영상에 모션이 잡힐 경우 
    if left_hand_landmarks or right_hand_landmarks: 
        motion_location = []
        
        # 왼손 랜드마크
        if left_hand_landmarks:
            for left_lm in left_hand_landmarks.landmark:
                motion_location.append([left_lm.x, left_lm.y, left_lm.z])
        else:
            for i in range(21):
                motion_location.append([0, 0, 0])
        
        # 오른손 랜드마크
        if right_hand_landmarks:
            for right_lm in right_hand_landmarks.landmark:
                motion_location.append([right_lm.x, right_lm.y, right_lm.z])
        else:
            for i in range(21):
                motion_location.append([0, 0, 0])
        
        return motion_location
    

In [10]:
def make_tensor_and_video(input_video_path, output_tensor_path, output_video_path, video_save=False):
    
    # Prepare DrawingSpec
    mp_drawing = mp.solutions.drawing_utils 
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

    # Config holistic
    mp_holistic = mp.solutions.holistic
    holistic = mp_holistic.Holistic(
        min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
    # 영상 가져오기
    cap = cv2.VideoCapture(input_video_path)

    # 영상 저장 1
    if video_save:
        fourcc = cv2.VideoWriter_fourcc(*'MP4V') # 영상 포맷
        out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (1280,720)) # 비디오 경로, 영상 포맷, 초당 프레임, width*height 

    # 영상 width, height 설정
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    # 각 요소(왼손, 오른손, 얼굴, 포즈) 좌표 저장 리스트
    hand_list = []
    
    while cap.isOpened():

        success, image = cap.read() 

        if not success: # 동영상 끝
            break

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = holistic.process(image)

        # Draw landmark annotation on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
          image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(
          image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        
        # 각 랜드마크를 리스트로 변환 후 리스트에 저장
        landmark = convert_landmark_to_tensor(results.left_hand_landmarks, results.right_hand_landmarks)
        if landmark:
            hand_list.append(landmark)
        
        # 영상 저장 2 (실질적인 영상 쓰기)
        if video_save:
            out.write(image)

    # 텐서로 변환
    hand_tensor = torch.FloatTensor(hand_list)
        
    # 텐서를 저장할 디렉토리 생성
    if not(os.path.isdir(output_tensor_path)):
        os.makedirs(output_tensor_path)
        
    # 텐서 파일 저장
    torch.save(hand_tensor, os.path.join(output_tensor_path, "hand.pt"))

    if video_save:
        # 동영사을 저장할 디렉토리 생성
        if not(os.path.isdir(output_video_path)):
            os.mkdir(output_video_path)
        
        # 영상 저장 3
        if video_save:
            out.release()

    holistic.close()
    cap.release()
    cv2.destroyAllWindows()

In [11]:
# 여러 디렉토리 생성 함수
def mkdirs(file_path_list: list, verbose=True):
    # ex) file_path_list= [".", "output", "images"]
    
    file_path = ""  # 생성할 디렉토리
    for path in file_path_list:
        file_path = os.path.join(file_path, path)
        
        # 디렉토리가 없다면 생성
        if not(os.path.isdir(file_path)):
            # 디렉토리 생성 안내 문구 출력
            if verbose:
                print("No {}".format(file_path))
                print("Make directory {}".format(file_path))
                print()
                
            # 디렉토리 생성
            os.mkdir(file_path)

In [12]:
def convert_videos_to_tensor(input_video_path_list=[".", "videos"],
                             output_tensor_path_list=[".", "output", "tensor"],
                             output_video_path_list=[".", "output", "video"],
                             videos_save=False):
    
    # csv파일 로드
    """
    csv1 = pd.read_csv("output\csv\KETI-2017-SL-0_10480-v2_1.csv")
    csv2 = pd.read_csv("output\csv\KETI-2018-SL-Annotation-v1.csv")

    dir_names = pd.concat([csv1['파일명'], csv2['파일명']], ignore_index=True)
    dir_names = dir_names[:43492]

    kors = pd.concat([csv1['한국어'], csv2['한국어']], ignore_index=True)
    kors = kors[:43492]
    """
    
    # 디렉토리 풀내임 생성
    input_video_path = os.path.join(*input_video_path_list)
    output_tensor_path = os.path.join(*output_tensor_path_list)
    output_video_path = os.path.join(*output_video_path_list)

    # 안내 문구 출력
    print("{:20}{}".format("input_video_path: ", input_video_path))
    print("{:20}{}".format("output_tensor_path: ", output_tensor_path))
    print("{:20}{}".format("videos_save: ", str(videos_save)))
    
    if videos_save:
        print("{:20}{}".format("output_video_path: ", output_video_path))
    print()
    
    # 디렉토리 생성
    mkdirs(input_video_path_list)
    mkdirs(output_tensor_path_list)
    if videos_save:
        mkdirs(output_video_path_list)

    # 비디오 폴더
    dir_video = os.listdir(input_video_path)
    dir_video.sort()
    
    # 각 비디오 폴더 순환
    for video_folder in dir_video:
        print("{:20}{}".format("current_video_folder: ", video_folder))
        final_input_video_path = input_video_path+"\\"+video_folder
        videos = os.listdir(final_input_video_path)
        
        for i, video in enumerate(videos):
                make_tensor_and_video(os.path.join(input_video_path, video_folder, video),
                                      os.path.join(output_tensor_path, video_folder, video[:-4]),  # [:-4]: 확장자 제거
                                      os.path.join(output_video_path, video),
                                      videos_save)
                if i%5==0:
                    print("{}/{} videos completed".format(i, len(videos)))
    

In [16]:
# 동영상 개장 3~5초 정도 소요
# ex) 동영상 50개: 4분 소요
convert_videos_to_tensor()

input_video_path:   .\videos
output_tensor_path: .\output\tensor
videos_save:        False

current_video_folder: num
0/8 videos completed
5/8 videos completed


In [14]:
# shape 확인
print(torch.load("./output/tensor/KETI_SL_0000000011/hand.pt"))

FileNotFoundError: [Errno 2] No such file or directory: './output/tensor/KETI_SL_0000000011/hand.pt'

현재 가장 바깥 리스트만 텐서로 바뀌고 텐서 내부는 전부 리스트임

---